# 如何添加消息历史记录
:::info 前提条件
本指南假设您熟悉以下概念：- [链式运行](/docs/how_to/sequence/)- [提示模板](/docs/concepts/prompt_templates)- [聊天消息](/docs/concepts/messages)- [LangGraph 持久化](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
:::
:::note
注意
本指南先前涵盖了 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象概念。您可以在 [v0.2 版本文档](https://python.langchain.com/v0.2/docs/how_to/message_history/) 中查看该版本的指南。
截至 LangChain v0.3 版本发布，我们建议 LangChain 用户利用 [LangGraph 持久化功能](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 集成到新的 LangChain 应用中。
如果你的代码已经在使用 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，那么你**无需**做任何更改。我们近期没有计划弃用此功能，因为它适用于简单的聊天应用，并且任何使用 `RunnableWithMessageHistory` 的代码将继续按预期工作。
请参阅[如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/) 获取更多详情。:::
在构建聊天机器人时，将对话状态传入和传出链是至关重要的。LangGraph 内置了持久化层，支持将链状态自动保存在内存中，或外部后端（如 SQLite、Postgres 或 Redis）。具体细节可查阅 LangGraph 的[持久化文档](https://langchain-ai.github.io/langgraph/how-tos/persistence/)。
在本指南中，我们将演示如何通过将任意LangChain可运行对象封装至一个极简的LangGraph应用中，为其添加持久化功能。这种方法能持久保存消息历史记录及链状态的其他元素，从而简化多轮交互应用的开发流程。同时，该方案支持多线程运行，使得单个应用能够分别与多个用户进行独立交互。
## 安装设置
让我们初始化一个聊天模型：
import ChatModelTabs from "@theme/ChatModelTabs";
<ChatModelTabs
customVarName="llm"/>

In [1]:
# | output: false
# | echo: false

# import os
# from getpass import getpass

# os.environ["ANTHROPIC_API_KEY"] = getpass()
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

## 示例：消息输入
为[聊天模型](/docs/concepts/chat_models)添加记忆功能提供了一个简单的示例。聊天模型接收消息列表作为输入并输出一条消息。LangGraph内置了可用于此目的的`MessagesState`状态管理工具。
以下，我们将：1. 将图状态定义为一个消息列表；2. 向图中添加一个调用聊天模型的单一节点；3. 使用内存检查点编译图，以便在运行之间存储消息。
:::信息
LangGraph 应用程序的输出是其[状态](https://langchain-ai.github.io/langgraph/concepts/low_level/)。这可以是任何 Python 类型，但在本上下文中，它通常是一个与可运行对象模式匹配的 `TypedDict`。
好的，请提供需要翻译的英文内容，我会将其转换为标准的中文markdown格式，并保持原有结构一致。

In [2]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

当我们运行应用程序时，会传入一个包含 `thread_id` 的配置字典。该 ID 用于区分对话线程（例如不同用户之间的会话）。

In [3]:
config = {"configurable": {"thread_id": "abc123"}}

然后我们可以调用该应用程序：

In [4]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I help you today?


In [5]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob, as you introduced yourself at the beginning of our conversation.


请注意，不同线程的状态是相互独立的。如果我们向一个带有新 `thread_id` 的线程发出相同的查询，模型会表示它不知道答案：

In [6]:
query = "What's my name?"
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly.


## 示例：字典输入
LangChain 可运行对象通常通过单个 `dict` 参数中的不同键来接收多个输入。一个常见的例子是包含多个参数的提示模板。
此前我们的可运行对象是一个聊天模型，而现在我们将一个提示模板和一个聊天模型串联在一起。

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

runnable = prompt | llm

在此场景中，我们将图状态定义为包含以下参数（除消息历史外）。随后，我们按照与之前相同的方式定义一个单节点图。
请注意以下状态：- 对 `messages` 列表的更新将追加消息；- 对 `language` 字符串的更新将覆盖该字符串。

In [8]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    response = runnable.invoke(state)
    # Update message history with response:
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc345"}}

input_dict = {
    "messages": [HumanMessage("Hi, I'm Bob.")],
    "language": "Spanish",
}
output = app.invoke(input_dict, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.


## 管理消息历史记录
消息历史记录（以及应用程序状态的其他元素）可通过 `.get_state` 访问：

In [10]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: Spanish
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.


我们也可以通过 `.update_state` 来更新状态。例如，我们可以手动追加一条新消息：

In [11]:
from langchain_core.messages import HumanMessage

_ = app.update_state(config, {"messages": [HumanMessage("Test")]})

In [12]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: Spanish
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.
================================ Human Message =================================

Test


有关状态管理的详细信息（包括删除消息），请参阅 LangGraph 文档：- [如何删除消息](https://langchain-ai.github.io/langgraph/how-tos/memory/delete-messages/)- [如何查看和更新历史图状态](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/)